In [1]:
import pandas as pd
import itertools
import os
import random
from tqdm import tqdm
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from openai import OpenAI

/home/vhodlevskyi/miniconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3508: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
level_quality = 2  # Threshold for good/bad MCQs
data_dir = "raiforukraine-MARIA/data/data_for_finetuning/Training_mcqs/"
datasets = ['gemma_9b_distractor_quality', 'Llama8B_0.1_distractor_quality', 'Llama1b_distractor_quality']  # Add 'phi3.5' when available
OPENAI_KEY = os.getenv('OPENAI_KEY')
max_gpt4o_attempts = 3

In [4]:
client = OpenAI(api_key=OPENAI_KEY)

In [5]:
class MCQQuestion(BaseModel):
    question: str = Field(description="The multiple-choice question")
    option_a: str = Field(description="The first answer option labeled 'A'")
    option_b: str = Field(description="The second answer option labeled 'B'")
    option_c: str = Field(description="The third answer option labeled 'C'")
    option_d: str = Field(description="The fourth answer option labeled 'D'")
    correct_option: str = Field(description="This consists only a letter of the correct option")

mcq_parser = JsonOutputParser(pydantic_object=MCQQuestion)

In [6]:
DEFAULT_PROMPT = """You are an expert in creating high-quality medical multiple-choice questions.
Create a challenging medical multiple-choice question with exactly one correct answer and three plausible distractors.

The distractors (incorrect options) quality is extremely important and should meet these criteria:
- They should be plausible to most test-takers
- They should represent common misconceptions
- They should require deep understanding to eliminate
- They should not be obviously incorrect or unrelated to the question
- They should not be easy to eliminate for knowledgeable test-takers

The question should test medical knowledge and critical thinking skills.
"""

In [7]:
DISTRACTORS_QUALITY_PROMPT = """You are tasked to evaluate the quality of the distractors of a multiple-choice question (incorrect options) on a scale of 1-5, where:
1 = POOR: Implausible, obviously incorrect, or unrelated to the question
2 = BELOW AVERAGE: Easy to eliminate, lacks plausibility for knowledgeable test-takers  
3 = AVERAGE: Somewhat plausible but contains minor flaws that make it distinguishable
4 = GOOD: Plausible to most test-takers, represents common misconceptions
5 = EXCELLENT: Highly plausible, represents sophisticated misconceptions, requires deep understanding to eliminate
Provide only a numerical score from 1 to 5 the best represents the level of distractor quality"""


In [8]:
def create_prompt_chain(prompt_text=DEFAULT_PROMPT):
    prompt_template = PromptTemplate(
        template="{prompt}.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={
            "prompt": prompt_text,
            "format_instructions": mcq_parser.get_format_instructions(),
        },
    )
    model = ChatOpenAI(model="gpt-4o", temperature=0.5, api_key=OPENAI_KEY)
    
    chain = prompt_template | model | mcq_parser
    return chain

In [9]:
def call_openai_api(client, system_prompt, user_prompt, temp=0.5, max_completion_tokens=1):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            temperature=temp,
            max_tokens=max_completion_tokens,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [10]:
def generate_prompt_for_question(row,
                                question_col='question',
                                option_a_col='option_a',
                                option_b_col='option_b',
                                option_c_col='option_c',
                                option_d_col='option_d',
                                correct_option='correct_option',
                                include_options=True,
                                include_correct_option=True,
                                context_col=None):
    question_text = row[question_col]
    options = f"a) {row[option_a_col]}\nb) {row[option_b_col]}\nc) {row[option_c_col]}\nd) {row[option_d_col]}"
    correct_opt = row[correct_option]
    
    user_prompt_delimiter = "-----\n"
    user_prompt_question = f"Question:\n{question_text}\n"
    user_prompt_options = f"Options:\n{options}\n"
    user_prompt = user_prompt_delimiter + user_prompt_question
    if include_options:
        user_prompt += user_prompt_options
    if include_correct_option:
        correct_option_text = f"Correct option: {correct_opt}\n"
        user_prompt += correct_option_text
    user_prompt += user_prompt_delimiter
    
    if context_col is not None and context_col in row:
        mcq_context = f"""Context:\n-----\n{row[context_col]}\n-----\n"""
        user_prompt = mcq_context + user_prompt
    return user_prompt

In [11]:
def evaluate_distractor_quality(mcq_dict):
    # Convert dictionary to a pandas Series for use with generate_prompt_for_question
    row = pd.Series(mcq_dict)
    
    # Generate prompt for evaluation
    user_prompt = generate_prompt_for_question(
        row,
        question_col='question',
        option_a_col='option_a',
        option_b_col='option_b',
        option_c_col='option_c',
        option_d_col='option_d',
        correct_option='correct_option',
        include_options=True,
        include_correct_option=True
    )
    
    # Call API to evaluate
    quality_score = call_openai_api(
        client,
        DISTRACTORS_QUALITY_PROMPT,
        user_prompt,
        temp=0.5,
        max_completion_tokens=1
    )
    
    try:
        return int(quality_score.strip())
    except (ValueError, TypeError):
        print(f"Error parsing quality score: {quality_score}")
        return 0

In [12]:
def generate_good_mcq_with_gpt4o(question_text):
    try:
        # Create the chain
        chain = create_prompt_chain()
        
        # Generate MCQ using the DEFAULT_PROMPT
        # Just pass the question topic without additional instructions
        query = question_text
        generated_mcq = chain.invoke({"query": query})
        
        # Evaluate distractor quality
        quality = evaluate_distractor_quality(generated_mcq)
        
        return generated_mcq, quality
        
    except Exception as e:
        print(f"Error generating MCQ with GPT-4o: {e}")
        return None, 0

In [13]:
def create_mcq_text(mcq_dict):
    return (
        f"Question: {mcq_dict['question']}\n"
        f"a) {mcq_dict['option_a']}\n"
        f"b) {mcq_dict['option_b']}\n"
        f"c) {mcq_dict['option_c']}\n"
        f"d) {mcq_dict['option_d']}"
    )

# Function to create a pair dictionary
def create_pair(lisa_id, good_mcq, bad_mcq, good_source="", bad_source=""):
    good_text = create_mcq_text(good_mcq)
    bad_text = create_mcq_text(bad_mcq)
    
    return {
        "id": lisa_id,
        "chosen": good_text,
        "rejected": bad_text,
        "chosen_source": good_source,
        "rejected_source": bad_source
    }

In [14]:
print("Processing MCQ datasets...")

id2mcqs = dict()  # Key: dataset, Value: {id: mcq_dict}
ids = set()       # Set of all common IDs across datasets
good_mcqs = dict() # Key: id, Value: list of datasets with good MCQs
bad_mcqs = dict()  # Key: id, Value: list of datasets with bad MCQs

Processing MCQ datasets...


In [15]:
for dataset in datasets:
    try:
        file_path = os.path.join(data_dir, f"{dataset}.csv")
        df = pd.read_csv(file_path)
        # df = df.iloc[:50]
        
        # Extract MCQ content and ensure correct_option is included
        temp = df[['id', 'question', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_option']].set_index('id').T.to_dict()
        
        # Update set of common IDs
        if len(ids) == 0:
            ids = set(temp.keys())
        else:
            ids = ids.intersection(set(temp.keys()))
        
        # Store MCQs by dataset
        id2mcqs[dataset] = temp.copy()
        
        # Categorize MCQs by quality
        for idx, row in df[['id', 'distractor_quality']].iterrows():
            lisa_id, quality = row
            
            # Good MCQ
            if quality > level_quality:
                if lisa_id in good_mcqs:
                    good_mcqs[lisa_id].append(dataset)
                else:
                    good_mcqs[lisa_id] = [dataset]
            else:
                if lisa_id in bad_mcqs:
                    bad_mcqs[lisa_id].append(dataset)
                else:
                    bad_mcqs[lisa_id] = [dataset]
                    
    except Exception as e:
        print(f"Error processing dataset {dataset}: {e}")

print(f"Found {len(ids)} common IDs across all datasets")

Found 3163 common IDs across all datasets


In [19]:
for dataset in datasets:
    file_path = os.path.join(data_dir, f"{dataset}.csv")
    df = pd.read_csv(file_path)
    print(f"{dataset} quality distribution:")
    print(df['distractor_quality'].value_counts())

# Check how many IDs have all good or all bad MCQs
print(f"IDs with all good MCQs: {nb_good_only}")
print(f"IDs with all bad MCQs: {nb_bad_only}")
print(f"IDs with mixed quality MCQs: {len(ids) - nb_good_only - nb_bad_only}")

gemma_9b_distractor_quality quality distribution:
distractor_quality
4    1488
3    1177
2     257
1     150
5      97
Name: count, dtype: int64
Llama8B_0.1_distractor_quality quality distribution:
distractor_quality
1    1176
3     939
2     827
4     219
5       2
Name: count, dtype: int64
Llama1b_distractor_quality quality distribution:
distractor_quality
3    1246
4     957
2     560
1     388
5      18
Name: count, dtype: int64
IDs with all good MCQs: 746
IDs with all bad MCQs: 95
IDs with mixed quality MCQs: 2322


In [18]:
nb_good_only = 0
nb_bad_only = 0
nb_mixed = 0
nb_gpt4o_generated = 0
all_pairs = []

# Process each ID to create pairs
for idx in tqdm(ids, desc="Creating pairs"):
    bad_mcqs_list = []
    good_mcqs_list = []
    
    if idx in bad_mcqs:
        bad_datasets = bad_mcqs[idx]
        for dataset in bad_datasets:
            if idx in id2mcqs[dataset]:
                bad_mcqs_list.append((id2mcqs[dataset][idx], dataset))
    else:
        # All MCQs are good
        nb_good_only += 1
        continue
        
    if idx in good_mcqs:
        good_datasets = good_mcqs[idx]
        for dataset in good_datasets:
            if idx in id2mcqs[dataset]:
                good_mcqs_list.append((id2mcqs[dataset][idx], dataset))
        nb_mixed += 1
    else:
        # All MCQs are bad - try to generate a good one with GPT-4o
        nb_bad_only += 1
        
        # Get sample question from one of the bad MCQs
        sample_question = bad_mcqs_list[0][0]["question"]
        
        # Try to generate a good MCQ with GPT-4o (up to max_attempts times)
        for attempt in range(max_gpt4o_attempts):
            generated_mcq, quality = generate_good_mcq_with_gpt4o(sample_question)
            
            if generated_mcq and quality >= level_quality:  # Aim for quality 3 or higher
                good_mcqs_list.append((generated_mcq, "gpt-4o"))
                nb_gpt4o_generated += 1
                print(f"Successfully generated MCQ with quality score {quality} for ID {idx}")
                break
            else:
                print(f"Attempt {attempt+1}: Generated MCQ quality ({quality}) below threshold of {level_quality} for ID {idx}")
        
        # If we couldn't generate a good MCQ, skip this ID
        if not good_mcqs_list:
            print(f"Could not generate good MCQ for ID {idx} after {max_gpt4o_attempts} attempts. Skipping.")
            continue
    
    # Create all possible (good, bad) pairs
    for good_item, bad_item in itertools.product(good_mcqs_list, bad_mcqs_list):
        good_mcq, good_source = good_item
        bad_mcq, bad_source = bad_item
        
        pair = create_pair(idx, good_mcq, bad_mcq, good_source, bad_source)
        all_pairs.append(pair)

# Convert to DataFrame
pairs_df = pd.DataFrame(all_pairs)

Creating pairs:   4%|██▉                                                                     | 128/3163 [00:02<00:59, 50.74it/s]

Successfully generated MCQ with quality score 4 for ID OIC-182-03-B


Creating pairs:   5%|███▉                                                                    | 171/3163 [00:05<01:43, 29.02it/s]

Successfully generated MCQ with quality score 3 for ID OIC-177-02-A


Creating pairs:   7%|████▊                                                                   | 209/3163 [00:07<02:09, 22.76it/s]

...

Successfully generated MCQ with quality score 5 for ID OIC-287-05-A


Creating pairs:  99%|██████████████████████████████████████████████████████████████████████▎| 3130/3163 [04:47<00:03,  9.56it/s]

Successfully generated MCQ with quality score 3 for ID OIC-060-01-A


Creating pairs: 100%|███████████████████████████████████████████████████████████████████████| 3163/3163 [04:50<00:00, 10.90it/s]

Successfully generated MCQ with quality score 4 for ID OIC-040-01-A


In [21]:
pairs_df

,id,chosen,rejected,chosen_source,rejected_source
0,OIC-131-17-A,Question: Which of the following is NOT a pote...,Question: What is a priority etiological diagn...,gemma_9b_distractor_quality,Llama8B_0.1_distractor_quality
1,OIC-131-17-A,Question: Which of the following is NOT a pote...,Question: What is the primary etiological fact...,gemma_9b_distractor_quality,Llama1b_distractor_quality
2,OIC-340-11-A,Question: What is the main purpose of confirmi...,Question: Which imaging technique is most effe...,Llama1b_distractor_quality,gemma_9b_distractor_quality
3,OIC-340-11-A,Question: What is the main purpose of confirmi...,Question: What is the purpose of additional in...,Llama1b_distractor_quality,Llama8B_0.1_distractor_quality
4,OIC-091-12-A,Question: Which of the following is a characte...,Question: What are the three main causes of re...,gemma_9b_distractor_quality,Llama8B_0.1_distractor_quality
...,...,...,...,...,...
4924,OIC-040-01-A,Question: A 45-year-old woman presents with su...,Question: What is the main difference between ...,gpt-4o,Llama1b_distractor_quality
4925,OIC-152-06-B,Question: Which of the following heart conditi...,Question: What is the main difference between ...,gemma_9b_distractor_quality,Llama1b_distractor_quality
4926,OIC-152-06-B,Question: Which group of heart diseases is con...,Question: What is the main difference between ...,Llama8B_0.1_distractor_quality,Llama1b_distractor_quality
4927,OIC-249-03-B,Question: What is the main purpose of psychoso...,Question: Which of the following is NOT a psyc...,Llama1b_distractor_quality,gemma_9b_distractor_quality


In [125]:
# Statistics
print(f"Statistics:")
print(f"  IDs with all good MCQs: {nb_good_only}")
print(f"  IDs with all bad MCQs: {nb_bad_only}")
print(f"  IDs with mixed quality MCQs: {nb_mixed}")
print(f"  MCQs generated with GPT-4o: {nb_gpt4o_generated}")
print(f"  Total pairs created: {len(pairs_df)}")

# Split into training and evaluation datasets
eval_size = min(len(pairs_df) // 10, 200)  # 10% or max 200 samples for evaluation
eval_pairs = pairs_df.sample(n=eval_size, random_state=42)
train_pairs = pairs_df.drop(eval_pairs.index)

# Save to CSV
pairs_df.to_csv("all_mcq_pairs.csv", index=False)
train_pairs.to_csv("train_mcq_pairs.csv", index=False)
eval_pairs.to_csv("eval_mcq_pairs.csv", index=False)

print(f"Created {len(train_pairs)} training pairs and {len(eval_pairs)} evaluation pairs")
print("Files saved as all_mcq_pairs.csv, train_mcq_pairs.csv, and eval_mcq_pairs.csv")

In [128]:
print(f'{nb_good_only} of Lisa sheets ({nb_good_only/len(ids)*100}%) gives only good MCQs across all LLMs')
print(f'{nb_bad_only} of Lisa sheets ({nb_bad_only/len(ids)*100}%) gives only bad MCQs across all LLMs')

746 of Lisa sheets (23.5852039203288%) gives only good MCQs across all LLMs
95 of Lisa sheets (3.0034777110338284%) gives only bad MCQs across all LLMs
